In [ ]:
%pip install vit-keras
%pip install tensorflow-addons

In [ ]:
%pip install tensorflow

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
from sklearn import preprocessing
from pathlib import Path

In [ ]:
labels_train = []
labels_test = []

path_train = []
path_test = []

In [ ]:
train_path = r"solar_cell_EL_image\PVELAD\EL2021\trainval\JPEGImages"
test_path = r"solar_cell_EL_image\PVELAD\EL2021\test\JPEGImages"

In [ ]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Paths
train_path = r"solar_cell_EL_image\PVELAD\EL2021\trainval\JPEGImages"
annotations_dir = r"solar_cell_EL_image\PVELAD\EL2021\trainval\Annotations"

# Lists to store data
labels_train = []
path_train = []

def extract_labels_from_xml(xml_file_path):
    """Extract object labels from an XML annotation file."""
    if not os.path.exists(xml_file_path):
        return []  # If XML file does not exist, return empty list
    
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    return [obj.find('name').text for obj in root.findall('object')]

# Iterate through images and extract labels
for filename in os.listdir(train_path):
    if filename.endswith(".jpg"):
        xml_file_path = os.path.join(annotations_dir, filename.replace('.jpg', '.xml'))
        labels = extract_labels_from_xml(xml_file_path)

        labels_train.append(labels)
        path_train.append(os.path.join(train_path, filename))

# Apply MultiLabelBinarizer
mlb = MultiLabelBinarizer()
labels_train_bin = mlb.fit_transform(labels_train)

# Convert to DataFrame
df = pd.DataFrame(labels_train_bin, columns=mlb.classes_)
df.insert(0, "image_path", path_train)  # Add image path as first column

# Save to CSV
csv_output_path = "1multilabel_annotations.csv"
df.to_csv(csv_output_path, index=False)

print(f"Saved multi-label annotations to {csv_output_path}")
print(df.head())  # Show first few rows


In [ ]:
print(len(mlb.classes_))

In [ ]:
# %pip install transformers

from transformers import ViTFeatureExtractor, TFAutoModelForImageClassification
from PIL import Image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

# Load model & feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
model = TFAutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")

def load_image(path):
    img = Image.open(path).convert("RGB").resize((224, 224))
    return np.array(img)

def make_gradcam_heatmap(img_array, model, last_conv_layer_name="vit.encoder.layer.11.output"):
    grad_model = tf.keras.models.Model(
        [model.model.input],
        [model.model.get_layer(last_conv_layer_name).output, model.model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, tf.argmax(predictions[0])]
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0) / tf.reduce_max(heatmap)
    return heatmap.numpy()

def overlay_heatmap(heatmap, image, alpha=0.4):
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap_color * alpha + image
    return np.uint8(superimposed_img)

# Upload and process
image_paths = ["Picture1.png", "Picture2.png", "Picture3.png"]
for path in image_paths:
    raw_img = load_image(path)
    inputs = feature_extractor(images=raw_img, return_tensors="tf")
    heatmap = make_gradcam_heatmap(inputs["pixel_values"], model)
    overlay_img = overlay_heatmap(heatmap, raw_img)
    Image.fromarray(overlay_img).save("gradcam_" + path)


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Load and preprocess images
def load_and_preprocess_images(image_paths):
    images = []
    for img_path in image_paths:
        img = cv2.imread(img_path)  # Load image using OpenCV
        img = cv2.resize(img, (desired_width, desired_height))  # Resize image
        img = img.astype(np.float32) / 255.0  # Normalize pixel values
        images.append(img)
    return np.array(images)

# Define desired width and height for resizing images
desired_width = 224
desired_height = 224

# Load and preprocess images
x_train = load_and_preprocess_images(path_train)

# Split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, labels_train_bin, test_size=0.1, random_state=42)

print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_val shape:", x_val.shape)
print("y_val shape:", y_val.shape)


In [ ]:
# IMPORTANT: Restart the kernel after installing/uninstalling TensorFlow before running this cell!
import tensorflow as tf

# Check if TensorFlow can access GPUs
gpu_devices = tf.config.list_physical_devices('GPU')

if gpu_devices:
    print("Found GPU(s):")
    for gpu in gpu_devices:
        print(gpu)
else:
    print("No GPU(s) found. TensorFlow will use CPU.")


In [ ]:
%pip install tensorflow==2.14.0


In [ ]:
import tensorflow as tf
import keras

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

In [ ]:
%pip uninstall tensorflow-addons


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from vit_keras import vit
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from sklearn.utils.class_weight import compute_class_weight

# Load ViT model
vit_model = vit.vit_b16(
    image_size=224,
    activation="sigmoid",
    pretrained=False,
    include_top=True,
    pretrained_top=False,
    classes=12
)

# Compile model
optimizer = Adam(learning_rate=0.0001)
vit_model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy", AUC(name="auc")]
)

# Callbacks
checkpoint = ModelCheckpoint("ViT_best_weights.h5", save_best_only=True, monitor="val_loss", mode="min")
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, verbose=1)

# vit_model.summary()


In [ ]:
# Train
history = vit_model.fit(
    x_train, y_train,
    epochs=100,
    batch_size=16,
    validation_data=(x_val, y_val),
    callbacks=[checkpoint, lr_scheduler]
)

# Save training history
pd.DataFrame(history.history).to_csv("ViT_training_history.csv", index=False)
print("ViT training completed and history saved.")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the aesthetic style of the plots
sns.set(style='whitegrid')

def plot_training_history(history, save_path=None):
    # Extract data from history object
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)

    # Plot training and validation accuracy
    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b-o', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'r-o', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy', fontsize=16)
    plt.xlabel('Epochs', fontsize=14)
    plt.ylabel('Accuracy', fontsize=14)
    plt.ylim(0, 1)  # Set y-axis limits for accuracy
    plt.legend()
    plt.grid(True)

    # Plot training and validation loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b-o', label='Training Loss')
    plt.plot(epochs, val_loss, 'r-o', label='Validation Loss')
    plt.title('Training and Validation Loss', fontsize=16)
    plt.xlabel('Epochs', fontsize=14)
    plt.ylabel('Loss', fontsize=14)
    max_loss = max(max(loss), max(val_loss))
    plt.ylim(0, 0.5)  # Set y-axis limits for accuracy
    #plt.ylim(0, max_loss + 0.1 * max_loss)  # Set y-axis limits for loss with padding
    plt.legend()
    plt.grid(True)

    plt.tight_layout()

    # Save the plot if a save path is provided
    if save_path:
        plt.savefig(save_path, dpi=300)
    plt.show()

# Example usage
plot_training_history(history, save_path='vit_training_history.png')


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import AUC

# Load the MobileNetV2 model with pre-trained ImageNet weights, excluding the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(12, activation='sigmoid')(x)  # Assuming 12 classes with multi-label classification

# Create the complete model
mobilenet_model = Model(inputs=base_model.input, outputs=output)

# Compile the model
optimizer = Adam(learning_rate=0.0001)
mobilenet_model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(name='auc')]
)

# Callbacks
checkpoint = ModelCheckpoint('MobileNetV2_best_weights.h5', save_best_only=True, monitor='val_loss', mode='min')
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)

# mobilenet_model.summary()


In [ ]:
# Train
history_mn = mobilenet_model.fit(
    x_train, y_train,
    epochs=100,
    batch_size=16,
    validation_data=(x_val, y_val),
    callbacks=[checkpoint, lr_scheduler]
)

# Save training history
pd.DataFrame(history.history).to_csv("Mobilenet_training_history.csv", index=False)
print("ViT training completed and history saved.")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the aesthetic style of the plots
sns.set(style='whitegrid')

def plot_training_history(history, save_path=None):
    # Extract data from history object
    acc = history_mn.history['accuracy']
    val_acc = history_mn.history['val_accuracy']
    loss = history_mn.history['loss']
    val_loss = history_mn.history['val_loss']
    epochs = range(1, len(acc) + 1)

    # Plot training and validation accuracy
    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b-o', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'r-o', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy', fontsize=16)
    plt.xlabel('Epochs', fontsize=14)
    plt.ylabel('Accuracy', fontsize=14)
    plt.ylim(0, 1)  # Set y-axis limits for accuracy
    plt.legend()
    plt.grid(True)

    # Plot training and validation loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b-o', label='Training Loss')
    plt.plot(epochs, val_loss, 'r-o', label='Validation Loss')
    plt.title('Training and Validation Loss', fontsize=16)
    plt.xlabel('Epochs', fontsize=14)
    plt.ylabel('Loss', fontsize=14)
    max_loss = max(max(loss), max(val_loss))
    plt.ylim(0, 0.5)  # Set y-axis limits for accuracy
    #plt.ylim(0, max_loss + 0.1 * max_loss)  # Set y-axis limits for loss with padding
    plt.legend()
    plt.grid(True)

    plt.tight_layout()

    # Save the plot if a save path is provided
    if save_path:
        plt.savefig(save_path, dpi=300)
    plt.show()

# Example usage
plot_training_history(history, save_path='mn_training_history.png')


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import AUC

# Load the ResNet50 model with pre-trained ImageNet weights, excluding the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(12, activation='sigmoid')(x)  # Assuming 12 classes with multi-label classification

# Create the complete model
resnet_model = Model(inputs=base_model.input, outputs=output)

# Compile the model
optimizer = Adam(learning_rate=0.0001)
resnet_model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(name='auc')]
)

# Callbacks
checkpoint = ModelCheckpoint('ResNet50_best_weights.h5', save_best_only=True, monitor='val_loss', mode='min')
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)

# resnet_model.summary()


In [ ]:
# Train
history_rs = resnet_model.fit(
    x_train, y_train,
    epochs=100,
    batch_size=16,
    validation_data=(x_val, y_val),
    callbacks=[checkpoint, lr_scheduler]
)

# Save training history
pd.DataFrame(history.history).to_csv("Resnet_training_history.csv", index=False)
print("Resnet training completed and history saved.")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the aesthetic style of the plots
sns.set(style='whitegrid')

def plot_training_history(history, save_path=None):
    # Extract data from history object
    acc = history_rs.history['accuracy']
    val_acc = history_rs.history['val_accuracy']
    loss = history_rs.history['loss']
    val_loss = history_rs.history['val_loss']
    epochs = range(1, len(acc) + 1)

    # Plot training and validation accuracy
    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b-o', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'r-o', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy', fontsize=16)
    plt.xlabel('Epochs', fontsize=14)
    plt.ylabel('Accuracy', fontsize=14)
    plt.ylim(0, 1)  # Set y-axis limits for accuracy
    plt.legend()
    plt.grid(True)

    # Plot training and validation loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b-o', label='Training Loss')
    plt.plot(epochs, val_loss, 'r-o', label='Validation Loss')
    plt.title('Training and Validation Loss', fontsize=16)
    plt.xlabel('Epochs', fontsize=14)
    plt.ylabel('Loss', fontsize=14)
    max_loss = max(max(loss), max(val_loss))
    plt.ylim(0, 0.5)  # Set y-axis limits for accuracy
    #plt.ylim(0, max_loss + 0.1 * max_loss)  # Set y-axis limits for loss with padding
    plt.legend()
    plt.grid(True)

    plt.tight_layout()

    # Save the plot if a save path is provided
    if save_path:
        plt.savefig(save_path, dpi=300)
    plt.show()

# Example usage
plot_training_history(history, save_path='rs_training_history.png')


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import AUC

# Load the EfficientNetB0 model with pre-trained ImageNet weights, excluding the top layer
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(12, activation='sigmoid')(x)  # Assuming 12 classes with multi-label classification

# Create the complete model
efficientnet_model = Model(inputs=base_model.input, outputs=output)

# Compile the model
optimizer = Adam(learning_rate=0.0001)
efficientnet_model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(name='auc')]
)

# Callbacks
checkpoint = ModelCheckpoint('EfficientNetB0_best_weights.h5', save_best_only=True, monitor='val_loss', mode='min')
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)

# efficientnet_model.summary()


In [ ]:

# Train
history_ef = efficientnet_model.fit(
    x_train, y_train,
    epochs=100,
    batch_size=16,
    validation_data=(x_val, y_val),
    callbacks=[checkpoint, lr_scheduler]
)

# Save training history
pd.DataFrame(history.history).to_csv("EF_training_history.csv", index=False)
print("EF training completed and history saved.")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the aesthetic style of the plots
sns.set(style='whitegrid')

def plot_training_history(history, save_path=None):
    # Extract data from history object
    acc = history_ef.history['accuracy']
    val_acc = history_ef.history['val_accuracy']
    loss = history_ef.history['loss']
    val_loss = history_ef.history['val_loss']
    epochs = range(1, len(acc) + 1)

    # Plot training and validation accuracy
    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b-o', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'r-o', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy', fontsize=16)
    plt.xlabel('Epochs', fontsize=14)
    plt.ylabel('Accuracy', fontsize=14)
    plt.ylim(0, 1)  # Set y-axis limits for accuracy
    plt.legend()
    plt.grid(True)

    # Plot training and validation loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b-o', label='Training Loss')
    plt.plot(epochs, val_loss, 'r-o', label='Validation Loss')
    plt.title('Training and Validation Loss', fontsize=16)
    plt.xlabel('Epochs', fontsize=14)
    plt.ylabel('Loss', fontsize=14)
    max_loss = max(max(loss), max(val_loss))
    plt.ylim(0, 0.5)  # Set y-axis limits for accuracy
    #plt.ylim(0, max_loss + 0.1 * max_loss)  # Set y-axis limits for loss with padding
    plt.legend()
    plt.grid(True)

    plt.tight_layout()

    # Save the plot if a save path is provided
    if save_path:
        plt.savefig(save_path, dpi=300)
    plt.show()

# Example usage
plot_training_history(history, save_path='ef_training_history.png')


In [ ]:

import matplotlib.pyplot as plt
import tensorflow as tf

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Example usage (replace with your model/image/layer)
# img_tensor = np.expand_dims(img, axis=0)
# heatmap = make_gradcam_heatmap(img_tensor, model, 'last_conv_layer_name')
# plt.matshow(heatmap)
# plt.show()
